In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
batch_size = 128
total_words = 10000
max_review_len = 80
embedding_len = 100

In [ ]:
# imdb.load_data 함수 사용하여 IMDB 데이터셋 다운
# 파라미터 num_words는 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지 사용할지를 의미
# 등장 빈도 순위가 1 ~ 10000에 해당하는 단어만 사용
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words = total_words)

'''
전체 훈련 셋에서 각 샘플의 길이는 서로 다를수 있음
또한 각 문장은 단어 수가 제 각각
모델의 입력으로 사용하려면 모든 샘플 길이를 동일하게 맞추어야함
이를 자연어 처리에서는 패딩작업이라고 함, 보통 숫자 0을 넣어서 길이를 맞춤
케라스에서는 pad_sequence()를 사용
- 첫 번쨰 인자 : 패딩을 진행할 데이터
- maxlen : 모든 데이터에 대해 정규화 할 길이
'''
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen = max_review_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen = max_review_len)

'''
넘파이 배열을 Dataset으로 변환
변환하려는 전체 데이터를 메모리로 로딩해야 하므로 큰 용량의 메모리가 필요
메모리 문제 해결책은 Dataset의 from_generator 사용
이것은 필요할 떄만 파이썬 generator을 통해 가져옴
'''
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))

'''
train_data.shuffle(10000) : 
- 데이터셋을 임의로 섞음, buffer_size 지정
- 버퍼에서 임의로 샘플 뽑고, 뽑은 샘플은 다른 샘플로 대체
- 데이터셋의 크기에 비해 크거나 같은 버퍼 크기로 지정

bathch()
- 몇 개의 샘플로 가중치를 갱신할지 지정

drop_remainder
- 마지막 배치 크기를 무시하고 지정한 배치 크기 사용
'''
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder = True)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = test_data.batch(batch_size, drop_remainder = True)

print("X_train_shape,", X_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print("X_test_shape", X_test.shape)

sample = next(iter(test_data))
print(sample[0].shape)

17473536/17464789 [==============================] - 0s 0us/step
X_train_shape, (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
X_test_shape (25000, 80)
(128, 80)


# LSTM 셀 

In [ ]:
class LSTM_Build(tf.keras.Model):
    def __init__(self, units):
        super(LSTM_Build, self).__init__()

        self.state0 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units]), tf.zeros([batch_size, units])]

        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len,
        
                                                   input_length=max_review_len)
        
        '''
        - 첫 번쨰 인자 :  메모리 셀의 개수
        - dropout : 전체 가중치 중 50% 값을 0으로 설정하여 사용하지 않겠다는 의미
        '''
        self.RNNCell0 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.RNNCell1 = tf.keras.layers.LSTMCell(units, dropout=0.5)
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1):
            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)

        x = self.outlayer(out1)
        prob = tf.sigmoid(x)

        return prob

In [ ]:

units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 56s 202ms/step - loss: 0.4855 - accuracy: 0.7482
Epoch 2/4
195/195 [==============================] - 54s 277ms/step - loss: 0.3113 - accuracy: 0.8704 - val_loss: 0.3604 - val_accuracy: 0.8379
Epoch 3/4
195/195 [==============================] - 39s 198ms/step - loss: 0.2620 - accuracy: 0.8930
Epoch 4/4
195/195 [==============================] - 48s 245ms/step - loss: 0.2158 - accuracy: 0.9175 - val_loss: 0.4203 - val_accuracy: 0.8348


In [ ]:
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
loss=0.1328, accuracy: 95.7252%
테스트 데이터셋 평가...
loss=0.4203, accuracy: 83.4816%
시간: 247.36914801597595


# LSTM 계층

In [ ]:
class LSTM_Build(tf.keras.Model):

    def __init__(self, units):
        super(LSTM_Build, self).__init__()

        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len,
                                                   input_length=max_review_len)
        self.rnn = tf.keras.Sequential([
  
            # units : 네트워크의 층 수(출력 공간 차원)
            # return_sequences = True : 마지막 출력 또는 전체 순서를 반환
            # 이떄 return sequences = False는 마지막 셀에서 밀집층이 한번만 적용
            # unroll : 시간 순서에 따라 입력층과 은닉층에 대한 네트워크를 펼침
            # 메모리 사용률은 높을 수 있찌만 계속 속도는 빨라질수있음
            
            tf.keras.layers.LSTM(units, dropout=0.5, return_sequences=True,
                                 unroll=True),
            tf.keras.layers.LSTM(units, dropout=0.5, unroll=True)
        ])
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training=None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)

        return prob

In [ ]:
units = 64
epochs = 4
t0 = time.time()

model = LSTM_Build(units)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

model.fit(train_data, epochs=epochs, validation_data=test_data, validation_freq=2)

Epoch 1/4
195/195 [==============================] - 58s 206ms/step - loss: 0.4907 - accuracy: 0.7452
Epoch 2/4
195/195 [==============================] - 57s 295ms/step - loss: 0.3129 - accuracy: 0.8707 - val_loss: 0.3597 - val_accuracy: 0.8389
Epoch 3/4
195/195 [==============================] - 41s 208ms/step - loss: 0.2566 - accuracy: 0.8972
Epoch 4/4
195/195 [==============================] - 55s 282ms/step - loss: 0.2206 - accuracy: 0.9145 - val_loss: 0.3937 - val_accuracy: 0.8334


In [ ]:
print("훈련 데이터셋 평가...")
(loss, accuracy) = model.evaluate(train_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))
print("테스트 데이터셋 평가...")
(loss, accuracy) = model.evaluate(test_data, verbose=0)
print("loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가...
loss=0.1605, accuracy: 95.0761%
테스트 데이터셋 평가...
loss=0.3937, accuracy: 83.3413%
시간: 1501.2773988246918
